## install mecab on mac
```
brew install mecab mecab-ipadic  
pip install mecab-python3
```

## install neologd
```
git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
cd mecab-ipadic-neologd
./bin/install-mecab-ipadic-neologd -n
```

## get Japanese articles
livedoor ニュースコーパス

```
wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
tar xvzf ldcc-20140209.tar.gz
```

In [1]:
from urllib import request 
import logging
from pathlib import Path
import numpy as np
import tqdm
import re
import MeCab
from gensim import corpora, models

In [2]:
mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")

In [3]:
[line.split("\t") for line in mecab.parse("認めたくないものだな。自分自身の若さ故の過ちというものを。").split("\n")]

[['認め', 'ミトメ', '認める', '動詞-自立', '一段', '連用形'],
 ['たく', 'タク', 'たい', '助動詞', '特殊・タイ', '連用テ接続'],
 ['ない', 'ナイ', 'ない', '助動詞', '特殊・ナイ', '基本形'],
 ['もの', 'モノ', 'もの', '名詞-非自立-一般', '', ''],
 ['だ', 'ダ', 'だ', '助動詞', '特殊・ダ', '基本形'],
 ['な', 'ナ', 'な', '助詞-終助詞', '', ''],
 ['。', '。', '。', '記号-句点', '', ''],
 ['自分自身', 'ジブンジシン', '自分自身', '名詞-固有名詞-一般', '', ''],
 ['の', 'ノ', 'の', '助詞-連体化', '', ''],
 ['若さ故の過ち', 'ワカサユエノアヤマチ', '若さ故の過ち', '名詞-固有名詞-一般', '', ''],
 ['という', 'トイウ', 'という', '助詞-格助詞-連語', '', ''],
 ['もの', 'モノ', 'もの', '名詞-非自立-一般', '', ''],
 ['を', 'ヲ', 'を', '助詞-格助詞-一般', '', ''],
 ['。', '。', '。', '記号-句点', '', ''],
 ['EOS'],
 ['']]

In [4]:
[line.split("\t") for line in mecab.parse("今日もがんばるぞい！").split("\n")]

[['今日', 'キョウ', '今日', '名詞-副詞可能', '', ''],
 ['も', 'モ', 'も', '助詞-係助詞', '', ''],
 ['がん', 'ガン', 'がん', '名詞-一般', '', ''],
 ['ばる', 'バル', 'バル', '名詞-一般', '', ''],
 ['ぞい', 'ゾイ', 'ぞい', '名詞-接尾-一般', '', ''],
 ['！', '！', '！', '記号-一般', '', ''],
 ['EOS'],
 ['']]

In [5]:
[line.split("\t") for line in mecab.parse("今日も一日がんばるぞい！").split("\n")]

[['今日も一日がんばるぞい', 'キョウモイチニチガンバルゾイ', '今日も一日がんばるぞい', '名詞-固有名詞-一般', '', ''],
 ['！', '！', '！', '記号-一般', '', ''],
 ['EOS'],
 ['']]

In [6]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]

In [7]:
def tokenizer(text, pos=False):
    text = re.sub(r"http://.*\.com", "", text)
    text = re.sub(r"Sports Watch|売れ筋チェック", "", text)
    text = text.lower()
    l = [line.split("\t") for line in mecab.parse(text).split("\n")]
    res = [i[0] if not pos else (i[0],i[3]) for i in l 
                   if len(i) >=4 
                       and ("名詞" in i[3] or "動詞" in i[3] or "形容詞" in i[3] )
                       and "数" not in i[3] and "助動詞" not in i[3] and "助詞" not in [3]
                       and  not re.search("[0-9]", i[0])
                       and i[0] not in stopwords
            ]
    return res

In [8]:
re.sub(r"http://.*\.com", "", "http://bb.com aaaaa")

' aaaaa'

In [9]:
tokenizer("認めたくないものだな。自分自身の若さ故の過ちというものを。")

['認め', '自分自身', '若さ故の過ち']

In [10]:
tokenizer("認めたくないものだな。自分自身の若さ故の過ちというものを。", pos=True)

[('認め', '動詞-自立'), ('自分自身', '名詞-固有名詞-一般'), ('若さ故の過ち', '名詞-固有名詞-一般')]

In [11]:
doc_path = "./text/"
doc_dir = Path(doc_path)
dirs = [i for i in doc_dir.iterdir() if i.is_dir()]
dirs

[PosixPath('text/movie-enter'),
 PosixPath('text/it-life-hack'),
 PosixPath('text/kaden-channel'),
 PosixPath('text/topic-news'),
 PosixPath('text/livedoor-homme'),
 PosixPath('text/peachy'),
 PosixPath('text/sports-watch'),
 PosixPath('text/dokujo-tsushin'),
 PosixPath('text/smax')]

In [12]:
articles = [a for categ in dirs for a in categ.iterdir()]

In [13]:
num_docs = len(articles)
num_docs

7376

In [14]:
def read_doc(doc_id):
    with articles[doc_id].open() as f:
        print(f.read())

In [15]:
read_doc(5000)

http://news.livedoor.com/article/detail/5613985/
2011-06-08T08:00:00+0900
【Sports Watch】不振続く松井秀に「マイナーでのプレーを余儀なくされる」
日本時間6日のレッドソックス戦では、アスレチックス・松井秀喜に22打席ぶりのヒットが飛び出した。しかし、すでに自己ワーストの19打席無安打を超えるなど、打率は.213と低迷、苦しい日々が続いていることに変わりはない。

そんな折、7日発売の「週刊アサヒ芸能」（6.16号）では、在米のスポーツジャーナリストが同誌に対し、“背水”松井の周辺情報を伝えている。

「キャンプ時から近年になく、膝の状態はよかったのですが、松井のスロースターターぶりは例年どおりでした。夏場には打撃が復調することも見込まれ、慌てることもないでしょうが、5月に入ってチームが低迷するや、期待が高かっただけに他の選手に増して、DHに入る松井への風当たりが厳しくなってしまった」という在米ジャーナリスト。

今後の展望については、「地元紙記者は『ビリー（GM）がシーズン終了まで松井を置いておくことはありえない』と断言していました。トレード期限の7月30日までに、松井で商売するためにトレードを画策するでしょうね。でも、今の成績では高年俸の松井をプレーオフの切り札として獲得する球団も見当たらない。『日本ではやらない』と語っていた松井ですから、このままでは元レッドソックス・岡島秀樹(35)同様にマイナーでのプレーを余儀なくされるでしょう」と述べている。

・週刊アサヒ芸能 ［ライト版］＜デジタル＞（PC版）



In [16]:
def get_bow(doc_id):
    with articles[doc_id].open() as f:
        f.readline()
        f.readline()
        return tokenizer(f.read())

In [17]:
get_bow(5000)[:10]

['不振', '続く', '松井秀', 'マイナー', 'プレー', '余儀なく', 'さ', 'れる', '日本時間', 'レッドソックス']

In [18]:
docs = []
for a in tqdm.tqdm(articles):
    with a.open() as f:
        f.readline()
        f.readline()
        docs.append(tokenizer(f.read()))

100%|██████████| 7376/7376 [00:31<00:00, 235.30it/s]


In [19]:
docs[5000][:10]

['不振', '続く', '松井秀', 'マイナー', 'プレー', '余儀なく', 'さ', 'れる', '日本時間', 'レッドソックス']

In [20]:
d = corpora.Dictionary(docs)

stopwordsで排除しきれなかった邪魔な単語を処理

In [21]:
d.filter_extremes(no_below=5, no_above=0.2)
# nob_below以下の個数の単語を無視
# no_aboveの割合以上に出てくる単語を無視

d.compactify()
# idを振り直してコンパクトにする。

In [22]:
dic_num = len(d)
dic_num

24291

In [23]:
d[1000]

'長い'

In [24]:
d.doc2bow(docs[5000][:20])

[(476, 1),
 (531, 1),
 (1068, 1),
 (1391, 1),
 (3111, 1),
 (3956, 1),
 (4660, 1),
 (5003, 1),
 (7861, 1),
 (10372, 1),
 (13890, 1),
 (16620, 1),
 (18487, 1),
 (22823, 1),
 (23686, 1),
 (23727, 1),
 (23735, 1)]

(id, num in a doc)

In [25]:
d[1309] # get word from id

'若者たち'

In [26]:
corpus = [d.doc2bow(w) for w in docs]

In [27]:
corpus_words = sum(count for doc in corpus for id, count in doc)
corpus_words

1433159

In [28]:
num_topics = 10

In [29]:
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [30]:
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=d, num_topics=num_topics, passes=10)

using symmetric alpha at 0.1
using symmetric eta at 0.1
using serial LDA version on this node
running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 7376 documents, updating model once every 2000 documents, evaluating perplexity every 7376 documents, iterating 50x with a convergence threshold of 0.001000
PROGRESS: pass 0, at document #2000/7376
merging changes from 2000 documents into a model of 7376 documents
topic #2 (0.100): 0.007*"映画" + 0.005*"公開" + 0.004*"google" + 0.003*"画面" + 0.003*"ロゴ" + 0.003*"監督" + 0.002*"サービス" + 0.002*"得" + 0.002*"pc" + 0.002*"表示"
topic #3 (0.100): 0.005*"映画" + 0.004*"画面" + 0.003*"公開" + 0.003*"機能" + 0.003*"監督" + 0.003*"アプリ" + 0.003*"シリーズ" + 0.003*"作品" + 0.003*"見る" + 0.003*"pc"
topic #0 (0.100): 0.008*"映画" + 0.006*"公開" + 0.005*"大" + 0.004*"監督" + 0.004*"作品" + 0.004*"本作" + 0.003*"機能" + 0.003*"容量" + 0.003*"製品" + 0.003*"写真"
topic #4 (0.100): 0.010*"映画" + 0.006*"公開" + 0.004*"pc" + 0.004*"作品" + 0.003*"画面" + 0.003*"表示" + 0.003*"製品

topic #9 (0.100): 0.013*"スマートフォン" + 0.012*"対応" + 0.012*"d" + 0.010*"機能" + 0.009*"搭載" + 0.009*"ソフトウェア" + 0.007*"s" + 0.006*"利用" + 0.006*"更新" + 0.006*"nttドコモ"
topic #2 (0.100): 0.011*"結婚" + 0.006*"相手" + 0.005*"恋愛" + 0.005*"思う" + 0.004*"男性" + 0.004*"気持ち" + 0.003*"子供" + 0.003*"夫" + 0.003*"選手" + 0.003*"くれ"
topic #6 (0.100): 0.006*"男性" + 0.005*"仕事" + 0.005*"独女" + 0.004*"好き" + 0.004*"思う" + 0.004*"くれ" + 0.004*"てる" + 0.003*"氏" + 0.003*"言わ" + 0.003*"女子"
topic diff=0.587142, rho=0.386680
PROGRESS: pass 2, at document #4000/7376
merging changes from 2000 documents into a model of 7376 documents
topic #4 (0.100): 0.016*"画面" + 0.015*"表示" + 0.011*"更新" + 0.009*"設定" + 0.008*"アプリ" + 0.007*"facebook" + 0.007*"twitter" + 0.006*"入力" + 0.006*"pc" + 0.005*"ファイル"
topic #6 (0.100): 0.005*"仕事" + 0.005*"男性" + 0.004*"声" + 0.004*"ネット掲示板" + 0.004*"てる" + 0.004*"氏" + 0.004*"番組" + 0.003*"思う" + 0.003*"好き" + 0.003*"発言"
topic #8 (0.100): 0.017*"max" + 0.013*"エスマックス" + 0.011*"s" + 0.008*"スマートフォン" + 0.007*"サービス" + 0.007*"対

topic #8 (0.100): 0.016*"max" + 0.014*"エスマックス" + 0.014*"s" + 0.014*"スマートフォン" + 0.009*"向け" + 0.008*"android" + 0.008*"サービス" + 0.008*"対応" + 0.007*"iphone" + 0.007*"製品"
topic #6 (0.100): 0.006*"男性" + 0.006*"仕事" + 0.005*"声" + 0.004*"てる" + 0.004*"ネット掲示板" + 0.004*"好き" + 0.004*"思う" + 0.003*"くれ" + 0.003*"発言" + 0.003*"番組"
topic #3 (0.100): 0.008*"転職" + 0.006*"年収" + 0.005*"livedoor" + 0.004*"企業" + 0.004*"会社" + 0.004*"仕事" + 0.004*"サービス" + 0.004*"必要" + 0.004*"ビデオsalon" + 0.003*"テレビ"
topic #1 (0.100): 0.021*"映画" + 0.010*"公開" + 0.006*"作品" + 0.005*"本作" + 0.004*"ドラマ" + 0.004*"いく" + 0.004*"主人公" + 0.003*"演じる" + 0.003*"dvd" + 0.003*"物語"
topic diff=0.263680, rho=0.339266
PROGRESS: pass 4, at document #6000/7376
merging changes from 2000 documents into a model of 7376 documents
topic #8 (0.100): 0.015*"max" + 0.014*"s" + 0.014*"スマートフォン" + 0.013*"エスマックス" + 0.009*"向け" + 0.008*"android" + 0.008*"サービス" + 0.008*"対応" + 0.007*"iphone" + 0.007*"モデル"
topic #4 (0.100): 0.019*"アプリ" + 0.017*"画面" + 0.015*"表示" + 0.010*"

merging changes from 2000 documents into a model of 7376 documents
topic #1 (0.100): 0.021*"映画" + 0.008*"公開" + 0.005*"作品" + 0.005*"本作" + 0.004*"ドラマ" + 0.004*"いく" + 0.004*"主人公" + 0.003*"シーン" + 0.003*"姿" + 0.003*"演じる"
topic #4 (0.100): 0.025*"アプリ" + 0.017*"画面" + 0.014*"表示" + 0.010*"設定" + 0.009*"twitter" + 0.009*"機能" + 0.007*"facebook" + 0.007*"便利" + 0.007*"iphone" + 0.006*"登録"
topic #5 (0.100): 0.043*"ご" + 0.023*"プレゼント" + 0.019*"肌" + 0.016*"応募" + 0.016*"ください" + 0.014*"当選" + 0.014*"キャンペーン" + 0.009*"いただき" + 0.009*"賞品" + 0.008*"発送"
topic #0 (0.100): 0.009*"映画" + 0.009*"作品" + 0.008*"監督" + 0.006*"公開" + 0.006*"開催" + 0.005*"イベント" + 0.005*"写真" + 0.005*"大" + 0.005*"ファン" + 0.004*"世界"
topic #2 (0.100): 0.010*"結婚" + 0.007*"氏" + 0.006*"選手" + 0.005*"相手" + 0.004*"試合" + 0.004*"てる" + 0.004*"思う" + 0.003*"て" + 0.003*"声" + 0.003*"恋愛"
topic diff=0.249802, rho=0.305880
-7.941 per-word bound, 245.8 perplexity estimate based on a held-out corpus of 1376 documents with 359131 words
PROGRESS: pass 6, at document 

topic #8 (0.100): 0.018*"スマートフォン" + 0.015*"s" + 0.014*"エスマックス" + 0.014*"max" + 0.011*"向け" + 0.010*"android" + 0.008*"サービス" + 0.008*"対応" + 0.008*"iphone" + 0.007*"利用"
topic diff=0.222005, rho=0.280739
-7.918 per-word bound, 241.9 perplexity estimate based on a held-out corpus of 1376 documents with 359131 words
PROGRESS: pass 8, at document #7376/7376
merging changes from 1376 documents into a model of 7376 documents
topic #3 (0.100): 0.007*"転職" + 0.004*"年収" + 0.004*"企業" + 0.004*"必要" + 0.004*"サービス" + 0.004*"会社" + 0.004*"テレビ" + 0.003*"デジ" + 0.003*"現在" + 0.003*"デジタル"
topic #6 (0.100): 0.010*"男性" + 0.008*"仕事" + 0.006*"独女" + 0.005*"好き" + 0.005*"くれ" + 0.005*"思う" + 0.004*"しれ" + 0.004*"しまう" + 0.004*"言わ" + 0.003*"聞い"
topic #8 (0.100): 0.022*"エスマックス" + 0.021*"s" + 0.021*"max" + 0.020*"スマートフォン" + 0.013*"向け" + 0.011*"android" + 0.008*"対応" + 0.008*"サービス" + 0.007*"利用" + 0.007*"端末"
topic #7 (0.100): 0.006*"人気" + 0.004*"女子" + 0.004*"アイテム" + 0.003*"写真" + 0.003*"peachy" + 0.003*"効果" + 0.003*"store" + 0.

In [31]:
sorted(lda.get_document_topics(corpus[5000]), key=lambda t:t[1], reverse=True)

[(2, 0.68815649), (7, 0.22667658), (6, 0.044827722), (3, 0.034397721)]

In [ ]:
def get_topic_words(topic_id):
    for t in lda.get_topic_terms(topic_id):
        print("{}: {}".format(d[t[0]], t[1]))

In [ ]:
for t in range(num_topics):
    print("Topic # ",t)
    get_topic_words(t)
    print("\n")

## perplexity

In [ ]:
lda.log_perplexity(corpus)

Estimate the variational bound of documents from corpus: E_q[log p(corpus)] - E_q[log q(corpus)]

In [ ]:
bound = lda.bound(corpus)/corpus_words
bound #= lda.log_perplexity(corpus)

In [ ]:
perplexity = np.exp2(-bound ) # 2^(-bound per words)

## see on tensorboard

http://projector.tensorflow.org/  
ref: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Tensorboard_visualizations.ipynb


In [ ]:
all_topics = lda.get_document_topics(corpus, minimum_probability=0)

In [ ]:
with open('doc_lda_tensor.tsv','w') as w:
    for doc_topics in all_topics:
        for topics in doc_topics:
            w.write(str(topics[1])+ "\t")
        w.write("\n")    

In [ ]:
meta = [str(a).split("/") for a in articles]

In [ ]:
meta[0]

In [ ]:
with open('doc_lda_metadata.tsv','w') as w:
    w.write('Titles\tGenres\n')
    for m in meta:
        w.write("%s\t%s\n" % (m[1][:2], m[1]))

## 類似度

In [ ]:
doc_topics = np.array([lda[c] for c in corpus])

In [ ]:
doc_topics.shape

In [ ]:
dense = np.zeros( (len(doc_topics), num_topics), float)

for doc_id, t in enumerate(doc_topics):
    for topic_id, weight in t:
        dense[doc_id, int(topic_id)] = weight

例えば
$$
\text{doc_id(7376)}
\underbrace{
\begin{pmatrix}
0 & 0 & weight & \cdots & 0\\
0 & 0 & 0 & \cdots & 0\\
0 & weight & 0 & \cdots & 0\\
\vdots & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & 0
\end{pmatrix}
}_{\text{topics(20)}}
$$

In [ ]:
from scipy.spatial import distance
pairwise = distance.squareform(distance.pdist(dense, metric="cosine"))

$$
1 - \frac{u \cdot v}
         {{||u||}_2 {||v||}_2}
$$

In [ ]:
def similar_to(doc_id, num=5, distance=False):
    doc = pairwise[doc_id]
    ids = np.argsort(doc)[1: num+1] # the most similar is self.
    if distance:
        dist = np.sort(doc)[1: num+1]
        return [(x,y) for x, y in zip(ids, dist)]
    else:
        return ids

In [ ]:
read_doc(1000)

In [ ]:
similar_to(1000, num=10, distance=True)

In [ ]:
read_doc(988)

In [ ]:
read_doc(2580)

In [ ]:
from gensim import similarities

In [ ]:
doc_index = similarities.docsim.MatrixSimilarity(lda[corpus])

In [ ]:
c = corpus[5000]
vec_lda = lda[c]

In [ ]:
s = doc_index.get_similarities(vec_lda)

In [ ]:
s = sorted(enumerate(s), key=lambda t: t[1], reverse=True)

In [ ]:
s[1:11]

In [ ]:
similar_to(5000, num=10, distance=True)